In [1]:
import numpy as np
import pandas as pd
import spacy
import time
import pickle

In [2]:
start_time = time.time()
nlp = spacy.load('fr_core_news_md')
#temps de récupération du modèle
print("--- %s secondes ---" % (time.time() - start_time))


--- 9.54554295539856 secondes ---


In [3]:
nlp("Bonjour").vector[:3]

array([-0.8534,  0.7579, -1.6385], dtype=float32)

In [4]:
#taille des vecteurs 
len(nlp("Bonjour").vector)

300

In [5]:
nlp("Mais où est donc Ornicar?").vector[:3]

array([ 0.12045503, -0.7953717 , -0.97345334], dtype=float32)

In [6]:
v = []
for token in nlp("Mais où est donc Ornicar?"):
    v.append(token.vector[:3])
v = np.array(v)
np.mean(v,axis=0)

array([ 0.12045503, -0.7953717 , -0.97345334], dtype=float32)

In [16]:
nlp("Bonjour").vector[:5]

array([-0.8534,  0.7579, -1.6385,  1.2507,  1.1918], dtype=float32)

In [7]:
nlp('coronavirus').vector[:5]

array([ 0.70707 , -0.072746, -4.5744  , -0.55461 ,  0.92508 ],
      dtype=float32)

In [8]:
nlp('covid').vector[:5]

array([0., 0., 0., 0., 0.], dtype=float32)

In [7]:
nlp('covid-19').vector[:5]

array([0., 0., 0., 0., 0.], dtype=float32)

In [10]:
df = pd.read_csv('../set_train_simple/eqtype_train_test15k_.csv')

from sklearn.model_selection import train_test_split

train, test = train_test_split(df.copy(), test_size=(1/3), random_state=0)

In [11]:
vectors = []
for i,titre in enumerate (train["titre"]):
    if i % 100==0:
        print(i)
    vectors.append(nlp(str(titre)).vector)
train['vector']=vectors

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [12]:
vectors = []
for i,titre in enumerate (test["titre"]):
    if i % 100==0:
        print(i)
    vectors.append(nlp(str(titre)).vector)
test['vector']=vectors

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [13]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix , accuracy_score

from sklearn.neural_network import MLPClassifier



X_train = np.array(np.stack(train["vector"]))
y_train = train['type']

X_test = np.array(np.stack(test["vector"]))
y_test = test['type']

In [14]:
clf = MLPClassifier(random_state=1, max_iter=200, hidden_layer_sizes=50,alpha=0.01).fit(X_train, y_train)
clf.fit(X_train, y_train)


y_pred_train = clf.predict(X_train)

cm_train = confusion_matrix(y_train, y_pred_train)

accuracy = accuracy_score(y_train,y_pred_train)
print('Accuracy:',accuracy)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.9958


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [15]:
clf.classes_

array(['actu', 'parodique', 'people', 'satirique', 'science'], dtype='<U9')

In [16]:
cm_train

array([[1933,    6,    0,    7,   20],
       [   2, 1980,    0,    2,    2],
       [   0,    1, 2005,    0,    0],
       [   0,    0,    0, 2023,    0],
       [   2,    0,    0,    0, 2017]], dtype=int64)

In [17]:
y_pred_test = clf.predict(X_test)

cm_test = confusion_matrix(y_test, y_pred_test)

#accuracy_score(y_train,y_pred_train)
print('Accuracy:', accuracy_score(y_test,y_pred_test))

Accuracy: 0.675


In [18]:
cm_test

array([[502, 157,  58, 148, 169],
       [161, 650,  79,  65,  59],
       [ 44,  67, 831,  28,  24],
       [102,  42,  30, 704,  99],
       [112,  73,  22,  86, 688]], dtype=int64)

In [19]:
clf.predict((nlp('Incendit chez OVH, un beau nuage de données').vector).reshape(1, -1))

array(['science'], dtype='<U9')

In [20]:
clf.predict((nlp('Pouet pouet, camion!').vector).reshape(1, -1))

array(['actu'], dtype='<U9')

In [21]:
clf.predict((nlp('Loana : son incroyable changement physique en 20 photos').vector).reshape(1, -1))

array(['people'], dtype='<U9')

In [22]:
clf.predict((nlp('Le boson de Higgs mis en évidence au CERN').vector).reshape(1, -1))

array(['science'], dtype='<U9')

In [23]:
clf.predict((nlp('Le FC Nantes a encore gagné avec brio').vector).reshape(1, -1))

array(['parodique'], dtype='<U9')

In [24]:
#random forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=13, random_state=0).fit(X_train, y_train)
y_pred_train = clf.predict(X_train)

cm_train = confusion_matrix(y_train, y_pred_train)

accuracy = accuracy_score(y_train,y_pred_train)
print('Accuracy:',accuracy)

Accuracy: 0.9879


In [25]:
clf

RandomForestClassifier(max_depth=13, random_state=0)

In [26]:
y_pred_test = clf.predict(X_test)

cm_test = confusion_matrix(y_test, y_pred_test)

#accuracy_score(y_train,y_pred_train)
print('Accuracy:', accuracy_score(y_test,y_pred_test))

Accuracy: 0.6086


In [ ]:
#mtériques



In [27]:
test.loc[7602]['vector'].reshape(1,300)

array([[-1.35344791e+00, -1.89303398e+00,  1.93870413e+00,
         2.23322535e+00, -4.91074383e-01, -4.91520353e-02,
         1.73730016e+00,  3.58630002e-01, -1.68119752e+00,
         2.20197961e-01, -5.65930009e-01,  2.70967984e+00,
        -2.27078390e+00, -1.74688411e+00,  6.22440040e-01,
        -4.72168058e-01, -1.68732011e+00, -2.25898600e+00,
        -1.12077974e-01,  1.17536402e+00,  1.29848993e+00,
        -1.53820825e+00, -3.91060188e-02, -5.17317951e-01,
         1.42635986e-01,  1.36126649e+00, -8.44227970e-01,
        -5.27398568e-03,  1.98174798e+00, -1.26934397e+00,
        -1.64914966e+00, -2.54778814e+00, -2.06648207e+00,
        -1.35789990e+00,  2.63188791e+00,  1.38875699e+00,
         6.63442016e-01,  1.27122462e+00,  1.54493403e+00,
         3.85219991e-01,  8.49204063e-01,  3.69103968e-01,
         4.50657994e-01, -6.14460588e-01, -1.93850011e-01,
         9.05974030e-01, -2.18189192e+00, -1.77425170e+00,
        -9.78328586e-01, -1.69522059e+00,  3.28578055e-0

In [28]:
test[400:430]

,date,titre,type,vector
14514,2017-09-15,Fronde en France contre un médicament soignant...,science,"[-0.1119846, -0.25657687, -1.1866069, 2.074190..."
18,2019-10-18,La dune de Brétignolles-sur-mer résiste toujours,satirique,"[-0.13838333, 1.0892982, 0.089066625, 3.518083..."
3923,2017-07-20,Quelle audace ! Gigi Hadid ose le lipstick gri...,people,"[-0.6196311, -0.7183322, -1.431294, 1.20738, -..."
950,2018-12-19,Cimon se révolte,satirique,"[2.8937533, -3.7095997, -2.2908366, 1.48149, -..."
4084,2017-03-02,Rebelle ? Ivanka Trump porte une robe qui ne p...,people,"[-0.076619886, -1.7513213, -2.1070058, 1.62147..."
5909,2017-04-27,Économie : le FN va plus loin et promet le ret...,parodique,"[-0.41785848, -1.4213331, -1.3671702, 2.038365..."
3991,2017-08-08,"Jean Lassalle, la nouvelle star de l'Assemblée...",people,"[-0.6252943, 1.155895, 1.19398, 2.577023, -1.7..."
9942,2017-08-01,Un membre de l’UPR assassine un homme qui ne c...,parodique,"[-0.17134805, -3.2362707, -1.1137207, 2.065492..."
8517,2019-08-02,"Yann Mingard, une expo à nous glacier le sang",actu,"[0.93249214, -1.142941, -1.097996, 1.7002671, ..."
14606,2020-11-14,Un masque chirurgical peut-il être réutilisé ?,science,"[0.94788253, -2.3628361, -3.1249402, 2.8241937..."


In [29]:
clf.predict(np.stack(test[328:329]['vector']))

array(['actu'], dtype=object)

In [30]:
clf.predict(test.loc[9010]['vector'].reshape(1,300))

array(['people'], dtype=object)

In [31]:
tokens = nlp("chien chat cerise pomme reine roi covid")

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

chien True 29.814888 False
chat True 36.303852 False
cerise True 22.43536 False
pomme True 29.365248 False
reine True 30.181189 False
roi True 58.487267 False
covid False 0.0 True


# test Fasttext

In [32]:
import fasttext
ft = fasttext.load_model('./fasttext/cc.fr.300.bin')

In [33]:
ft.get_word_vector('a la')

array([ 0.01558981,  0.00724296,  0.00020723,  0.00859443,  0.00386667,
       -0.01916484,  0.02798465, -0.0090925 ,  0.02790334, -0.0085213 ,
        0.01516476, -0.00863314, -0.01517181,  0.00408668,  0.0042271 ,
       -0.00527015, -0.00155622, -0.00274517,  0.0151117 , -0.00397785,
        0.00153633,  0.01015168,  0.02503621,  0.00744972, -0.00630598,
       -0.01395096, -0.01072407, -0.02084028, -0.00318747, -0.01111878,
        0.01707964,  0.02662326,  0.02691772, -0.01092142, -0.00099457,
        0.00751985,  0.01087526, -0.03254809, -0.03660877,  0.00885346,
       -0.03155928, -0.00512531,  0.02507202, -0.014594  ,  0.0206846 ,
        0.12876317,  0.01242106, -0.01114142,  0.01382924, -0.0043252 ,
        0.03757777, -0.02352742, -0.01018777, -0.00198512,  0.00142792,
        0.03629615, -0.00570226, -0.01976028,  0.01170014, -0.00260037,
       -0.050543  , -0.01526435,  0.01144967,  0.00558585,  0.01775417,
       -0.02286035,  0.01177743, -0.02892347, -0.00720178, -0.03

In [34]:

("je suis un amis de la famille").split()

['je', 'suis', 'un', 'amis', 'de', 'la', 'famille']

In [35]:
ft.get_word_vector("je suis un amis de la famille ")[:3]

array([-0.00739406,  0.0079207 ,  0.00353705], dtype=float32)

In [36]:
v = []
for token in ("je suis un amis de la famille ").split():
    v.append(ft.get_word_vector(str(token))[:3])
v = np.array(v)
np.mean(v,axis=0)

array([ 0.00573167, -0.01054828,  0.02611363], dtype=float32)

In [37]:
ft.get_nearest_neighbors('covid')

[(0.46579504013061523, 'Leposaviq'),
 (0.436837375164032, 'Dovid'),
 (0.4279879033565521, 'Rodovid'),
 (0.40892016887664795, 'Leposavić'),
 (0.387155145406723,
  'babylone-2-0-le-groupe-facebook-secret-qui-diffuse-des-photos-volees-de-femmes-nues-482095.htmlLCI'),
 (0.3854500353336334, 'jahia'),
 (0.3773195147514343, 'WebChat'),
 (0.3767986595630646, 'GrodlLukáš'),
 (0.37524208426475525,
  'CrônicasEsdrasNeemiasEsterJóSalmosProvérbiosEclesiastesCânticosIsaíasJeremiasLamentaçõesEzequielDanielOséiasJoelAmósObadiasJonasMiquéiasNaumHabacuqueSofoniasAgeuZacariasMalaquiasNovo'),
 (0.37134307622909546, 'directory.eoportal.org')]

In [38]:
ft.get_nearest_neighbors('koi')

[(0.6743189692497253, 'pourkoi'),
 (0.6606318950653076, 'alor'),
 (0.620459258556366, 'qoi'),
 (0.6180223822593689, 'kelkun'),
 (0.6123362183570862, 'quoid'),
 (0.6016865968704224, 'kelkin'),
 (0.5949128866195679, 'fodrai'),
 (0.5885655879974365, 'jvoulé'),
 (0.5868099331855774, 'jvois'),
 (0.586654782295227, 'pkoi')]

In [39]:
ft.get_word_vector("je")[:3]

array([0.04253509, 0.1088779 , 0.18245952], dtype=float32)

In [40]:
vectors_ft = []
for i,titre in enumerate (train["titre"]):
    if i % 100==0:
        print(i)
    vectors_ft.append(ft.get_word_vector(str(titre)))
train['vector_ft']=vectors_ft

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [41]:
vectors_ft = []
for i,titre in enumerate (test["titre"]):
    if i % 100==0:
        print(i)
    vectors_ft.append(ft.get_word_vector(str(titre)))
test['vector_ft']=vectors_ft

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [42]:
X_train = np.array(np.stack(train["vector_ft"]))
y_train = train['type']

X_test = np.array(np.stack(test["vector_ft"]))
y_test = test['type']

In [43]:
clf = MLPClassifier(random_state=1, max_iter=300, 
                    hidden_layer_sizes=80,alpha=0.01,
                    learning_rate='invscaling',
                   learning_rate_init=0.001).fit(X_train, y_train)
clf.fit(X_train, y_train)


y_pred_train = clf.predict(X_train)

cm_train = confusion_matrix(y_train, y_pred_train)

accuracy = accuracy_score(y_train,y_pred_train)
print('Accuracy:',accuracy)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.6999


In [44]:
y_pred_test = clf.predict(X_test)

cm_test = confusion_matrix(y_test, y_pred_test)

#accuracy_score(y_train,y_pred_train)
print('Accuracy:', accuracy_score(y_test,y_pred_test))

Accuracy: 0.5464


In [45]:
train

,date,titre,type,vector,vector_ft
12838,2018-05-22,Les zones d'ombre du code source de Parcoursup,science,"[0.6533856, -0.5908422, 0.6471245, 1.6466993, ...","[-0.010792193, 0.009429373, 0.007772304, 0.003..."
4772,2018-06-25,"Oscar Vela (Elior Group), Serge Morelli (Axa),...",actu,"[-1.2111242, 2.027541, 2.1903155, 1.2944391, -...","[-0.0029092066, -0.000761234, 0.0058668675, -0..."
5342,2017-04-24,François Fillon dénonce un acharnement des Fra...,parodique,"[0.36201003, -1.409817, -1.4338468, 2.0844471,...","[-0.0006036326, -3.366894e-05, -0.0005136254, ..."
9333,2019-04-16,Un passeport syrien en parfait état retrouvé d...,parodique,"[1.0026183, -0.39407402, 0.01794928, 0.9272113...","[0.0021052142, -0.002624337, 0.0040472997, -0...."
6499,2018-01-15,Balle aux prisonniers : Il réapprend à vivre d...,parodique,"[1.784022, -1.2697986, -0.39021853, 1.225765, ...","[2.4496765e-05, 0.0074398676, -0.0056175417, -..."
...,...,...,...,...,...
13123,2017-11-13,L'Esma avertit qu'une cryptomonnaie peut perdr...,science,"[-0.35314548, -2.32449, -2.2268734, 2.1695452,...","[0.01671319, 0.0024664034, -0.00019427063, -0...."
3264,2020-06-25,Elizabeth II : ce geste de François Hollande q...,people,"[-0.48734412, -0.7927884, -0.8117989, 2.239425...","[0.0006343277, -0.0021176198, -0.0069427732, 0..."
9845,2019-02-17,Carlos Ghosn dénonce: « Je n’ai jamais rien vo...,parodique,"[-0.527139, 0.13341044, 0.4138379, 1.8070544, ...","[0.01503535, 0.0053510247, -0.0031554208, 0.01..."
10799,2020-07-13,"Photos: Enora Malagré, Nikos Aliagas, Karine L...",people,"[-1.3412815, 0.6179895, 0.37152445, 1.7095574,...","[-0.0032453146, -0.003009305, 0.006629204, -0...."


# test fastext 2 ( mean)

In [46]:
vectors_ft2 = []
for i,titre in enumerate (train["titre"]):
    v = []
    if i % 100==0:
        print(i)
    for token in str(titre).split():
        v.append(ft.get_word_vector(str(token)))
    vectors_ft2.append(np.mean(np.array(v),axis=0))
train['vector_ft2']=vectors_ft2

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [47]:
vectors_ft2 = []
for i,titre in enumerate (test["titre"]):
    v = []
    if i % 100==0:
        print(i)
    for token in str(titre).split():
        v.append(ft.get_word_vector(str(token)))
    vectors_ft2.append(np.mean(np.array(v),axis=0))
test['vector_ft2']=vectors_ft2

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [48]:
def to_vec_ft_mean(phrase):
    v=[]
    for token in str(phrase).split():
        v.append(ft.get_word_vector(str(token)))
    return (np.mean(np.array(v),axis=0))

In [49]:
to_vec_ft_mean('pouet pouet camion')

array([ 2.60838568e-02, -1.52932731e-02, -4.46465239e-03,  1.26332464e-02,
        1.63745545e-02,  6.25630990e-02,  1.03553712e-01, -2.82034073e-02,
       -5.37776612e-02, -4.79133427e-02, -1.10696899e-02, -3.41391154e-02,
        1.59671120e-02,  6.30219802e-02,  7.40174763e-03,  6.00499399e-02,
        9.64401439e-02, -3.46619263e-02,  8.78771469e-02,  5.51832020e-02,
        8.49776268e-02,  7.14409119e-03, -8.33845064e-02, -3.73713896e-02,
        4.07613888e-02,  1.25148250e-02, -6.03163503e-02, -2.87613720e-02,
        3.34168188e-02,  5.02338335e-05, -2.41815690e-02,  1.19423606e-02,
        1.02573708e-02, -2.47456413e-03, -1.72367580e-02,  3.29399817e-02,
        1.49585763e-02, -5.53366244e-02, -3.93437818e-02, -3.19089182e-02,
       -6.62180111e-02,  1.31187001e-02,  2.22389892e-04, -1.04651660e-01,
        3.88013236e-02, -1.87178031e-02, -8.00355747e-02,  2.06979178e-03,
        6.19642176e-02, -3.19378972e-02,  9.62895975e-02,  1.46429986e-04,
        1.93250980e-02,  

In [50]:
train

,date,titre,type,vector,vector_ft,vector_ft2
12838,2018-05-22,Les zones d'ombre du code source de Parcoursup,science,"[0.6533856, -0.5908422, 0.6471245, 1.6466993, ...","[-0.010792193, 0.009429373, 0.007772304, 0.003...","[-0.012687968, 0.018681016, 0.016573738, -0.02..."
4772,2018-06-25,"Oscar Vela (Elior Group), Serge Morelli (Axa),...",actu,"[-1.2111242, 2.027541, 2.1903155, 1.2944391, -...","[-0.0029092066, -0.000761234, 0.0058668675, -0...","[0.016807659, -0.013200439, 0.0013597086, -0.0..."
5342,2017-04-24,François Fillon dénonce un acharnement des Fra...,parodique,"[0.36201003, -1.409817, -1.4338468, 2.0844471,...","[-0.0006036326, -3.366894e-05, -0.0005136254, ...","[0.0023232333, 0.01988542, 0.007383062, -0.033..."
9333,2019-04-16,Un passeport syrien en parfait état retrouvé d...,parodique,"[1.0026183, -0.39407402, 0.01794928, 0.9272113...","[0.0021052142, -0.002624337, 0.0040472997, -0....","[0.022624873, -0.020203995, 0.020829935, -0.02..."
6499,2018-01-15,Balle aux prisonniers : Il réapprend à vivre d...,parodique,"[1.784022, -1.2697986, -0.39021853, 1.225765, ...","[2.4496765e-05, 0.0074398676, -0.0056175417, -...","[0.0031588525, 0.0203431, -0.014027253, -0.011..."
...,...,...,...,...,...,...
13123,2017-11-13,L'Esma avertit qu'une cryptomonnaie peut perdr...,science,"[-0.35314548, -2.32449, -2.2268734, 2.1695452,...","[0.01671319, 0.0024664034, -0.00019427063, -0....","[0.00460826, 0.03221346, -0.013482266, -0.0034..."
3264,2020-06-25,Elizabeth II : ce geste de François Hollande q...,people,"[-0.48734412, -0.7927884, -0.8117989, 2.239425...","[0.0006343277, -0.0021176198, -0.0069427732, 0...","[-0.04057615, 0.02025055, -0.03451099, -0.0227..."
9845,2019-02-17,Carlos Ghosn dénonce: « Je n’ai jamais rien vo...,parodique,"[-0.527139, 0.13341044, 0.4138379, 1.8070544, ...","[0.01503535, 0.0053510247, -0.0031554208, 0.01...","[-0.016100543, 0.06398363, 0.008644857, -0.017..."
10799,2020-07-13,"Photos: Enora Malagré, Nikos Aliagas, Karine L...",people,"[-1.3412815, 0.6179895, 0.37152445, 1.7095574,...","[-0.0032453146, -0.003009305, 0.006629204, -0....","[0.0038269216, 0.0058808355, 0.026493484, -0.0..."


In [51]:
X_train = np.array(np.stack(train["vector_ft2"]))
y_train = train['type']

X_test = np.array(np.stack(test["vector_ft2"]))
y_test = test['type']

In [52]:
clf = MLPClassifier(random_state=1, max_iter=200, hidden_layer_sizes=30,alpha=0.01).fit(X_train, y_train)
clf.fit(X_train, y_train)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(alpha=0.01, hidden_layer_sizes=30, random_state=1)

In [53]:
y_pred_train = clf.predict(X_train)

cm_train = confusion_matrix(y_train, y_pred_train)

accuracy = accuracy_score(y_train,y_pred_train)
print('Accuracy:',accuracy)

Accuracy: 0.7724


In [54]:
y_pred_test = clf.predict(X_test)

cm_test = confusion_matrix(y_test, y_pred_test)

#accuracy_score(y_train,y_pred_train)
print('Accuracy:', accuracy_score(y_test,y_pred_test))

Accuracy: 0.6856


In [55]:
clf = MLPClassifier(random_state=1, max_iter=200, hidden_layer_sizes=(20,10),alpha=0.01).fit(X_train, y_train)
clf.fit(X_train, y_train)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(alpha=0.01, hidden_layer_sizes=(20, 10), random_state=1)

In [56]:
y_pred_train = clf.predict(X_train)

cm_train = confusion_matrix(y_train, y_pred_train)

accuracy = accuracy_score(y_train,y_pred_train)
print('Accuracy:',accuracy)

Accuracy: 0.7861


In [57]:
y_pred_test = clf.predict(X_test)

cm_test = confusion_matrix(y_test, y_pred_test)

#accuracy_score(y_train,y_pred_train)
print('Accuracy:', accuracy_score(y_test,y_pred_test))

Accuracy: 0.6786


In [58]:
# Saving model to disk
#pickle.dump(clf, open('../web_app/app/model1.pkl','wb'))

In [59]:
# Loading model to compare the results
#model = pickle.load(open('model1.pkl','rb'))
#print(model.predict([[2, 9, 6]]))

#model.predict(to_vec_ft_mean("L'énergie nucléaire est elle la plus propre?").reshape(1, -1))

In [60]:
#str(to_vec_ft_mean("L'énergie nucléaire est elle la plus propre?"))

In [84]:
def spacy_similarity(word, topn=10):
    word = nlp.vocab[str(word)]
    queries = [
      w for w in word.vocab if w.is_lower == word.is_lower and np.count_nonzero(w.vector)
    ]
    by_similarity = sorted(queries, key=lambda w: w.similarity(word), reverse=True)
    return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

In [84]:
spacy_similarity('mangeâmes', topn=5)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  import sys


[('aidants', 0.0),
 ('collectivités', 0.0),
 ('lacrymogènes', 0.0),
 ('coursiers', 0.0),
 ('qui', 0.0),
 ('printemps', 0.0)]

In [85]:
ft.get_nearest_neighbors('mangeâmes')[:5]

[(0.7825667858123779, 'partageâmes'),
 (0.765173077583313, 'échangeâmes'),
 (0.7586296200752258, 'mangeai'),
 (0.721340000629425, 'dirigeâmes'),
 (0.7204623818397522, 'discutâmes')]